In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import numpy.random as npr
import random

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import backend as K
from keras.optimizers import Adam
from keras_nlp.layers import PositionEmbedding

In [3]:
seed = 428

np.random.seed(seed)
tf.random.set_seed(seed)
random.seed(seed)

In [4]:
def bert_module(query, key, value, embed_dim, num_head, i):
    
    # Multi headed self-attention
    attention_output = layers.MultiHeadAttention(
        num_heads=num_head,
        key_dim=embed_dim // num_head,
        name="encoder_{}/multiheadattention".format(i)
    )(query, key, value, use_causal_mask=True)
    
    # Add & Normalize
    attention_output = layers.Add()([query, attention_output])  # Skip Connection
    attention_output = layers.LayerNormalization(epsilon=1e-6)(attention_output)
    
    # Feedforward network
    ff_net = keras.models.Sequential([
        layers.Dense(2 * embed_dim, activation='relu', name="encoder_{}/ffn_dense_1".format(i)),
        layers.Dense(embed_dim, name="encoder_{}/ffn_dense_2".format(i)),
    ])

    # Apply Feedforward network
    ffn_output = ff_net(attention_output)

    # Add & Normalize
    ffn_output = layers.Add()([attention_output, ffn_output])  # Skip Connection
    ffn_output = layers.LayerNormalization(epsilon=1e-6)(ffn_output)
    
    return ffn_output

In [5]:
def get_sinusoidal_embeddings(sequence_length, embedding_dim):
    position_enc = np.array([
        [pos / np.power(10000, 2. * i / embedding_dim) for i in range(embedding_dim)]
        if pos != 0 else np.zeros(embedding_dim)
        for pos in range(sequence_length)
    ])
    position_enc[1:, 0::2] = np.sin(position_enc[1:, 0::2])  # dim 2i
    position_enc[1:, 1::2] = np.cos(position_enc[1:, 1::2])  # dim 2i+1
    return tf.cast(position_enc, dtype=tf.float32)

In [6]:
N = 20 # vocab_size

vocabs = ['word_' + str(i) for i in range(N)]

vocab_map = {}
for i in range(len(vocabs)):
    vocab_map[vocabs[i]] = i
    
pairs = []

for i in vocabs:
    for j in vocabs:
        for k in vocabs:
            if i != j and i != k and j != k:
                pairs.append((i,j,k))
            
indicator = np.random.choice([0, 1], size=len(pairs), p=[0.5, 0.5])

pairs_train = [pairs[i] for i in range(len(indicator)) if indicator[i] == 1]
pairs_test = [pairs[i] for i in range(len(indicator)) if indicator[i] == 0]

In [7]:
sentences_train = []
sentences_number_train = []
sentences_test = []
sentences_number_test = []

for pair in pairs_train:
    sentences_train.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    sentences_test.append([pair[0], pair[1], pair[2], pair[0]])
    sentences_number_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = []
y_masked_labels_train = []
x_masked_test = []
y_masked_labels_test = []

for pair in pairs_train:
    x_masked_train.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_train.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
for pair in pairs_test:
    x_masked_test.append([vocab_map[pair[0]], vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    y_masked_labels_test.append([vocab_map[pair[1]], vocab_map[pair[2]], vocab_map[pair[0]]])
    
x_masked_train = np.array(x_masked_train)
y_masked_labels_train = np.array(y_masked_labels_train)
x_masked_test = np.array(x_masked_test)
y_masked_labels_test = np.array(y_masked_labels_test)

perm = np.random.permutation(len(x_masked_train))
x_masked_train = x_masked_train[perm]
y_masked_labels_train = y_masked_labels_train[perm]

In [8]:
embed_dim = 10
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
position_embeddings = PositionEmbedding(sequence_length=len(x_masked_train[0]))(word_embeddings)
encoder_output = word_embeddings + position_embeddings

for i in range(5):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

2024-05-22 07:04:06.397947: W tensorflow/tsl/platform/default/dso_loader.cc:66] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-05-22 07:04:06.397986: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-22 07:04:06.398006: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (gl3035.arc-ts.umich.edu): /proc/driver/nvidia/version does not exist
2024-05-22 07:04:06.398190: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/2000
1/1 - 4s - loss: 3.2596 - val_loss: 3.2190 - 4s/epoch - 4s/step
Epoch 2/2000
1/1 - 0s - loss: 3.2195 - val_loss: 3.1927 - 89ms/epoch - 89ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.1926 - val_loss: 3.1738 - 84ms/epoch - 84ms/step
Epoch 4/2000
1/1 - 0s - loss: 3.1731 - val_loss: 3.1591 - 82ms/epoch - 82ms/step
Epoch 5/2000
1/1 - 0s - loss: 3.1580 - val_loss: 3.1468 - 97ms/epoch - 97ms/step
Epoch 6/2000
1/1 - 0s - loss: 3.1449 - val_loss: 3.1362 - 94ms/epoch - 94ms/step
Epoch 7/2000
1/1 - 0s - loss: 3.1334 - val_loss: 3.1273 - 84ms/epoch - 84ms/step
Epoch 8/2000
1/1 - 0s - loss: 3.1236 - val_loss: 3.1202 - 82ms/epoch - 82ms/step
Epoch 9/2000
1/1 - 0s - loss: 3.1155 - val_loss: 3.1145 - 81ms/epoch - 81ms/step
Epoch 10/2000
1/1 - 0s - loss: 3.1089 - val_loss: 3.1101 - 81ms/epoch - 81ms/step
Epoch 11/2000
1/1 - 0s - loss: 3.1036 - val_loss: 3.1062 - 82ms/epoch - 82ms/step
Epoch 12/2000
1/1 - 0s - loss: 3.0988 - val_loss: 3.1026 - 81ms/epoch - 81ms/step
Epoch 13/2000
1/1 - 0s - loss

1/1 - 0s - loss: 2.7334 - val_loss: 2.7417 - 81ms/epoch - 81ms/step
Epoch 102/2000
1/1 - 0s - loss: 2.7270 - val_loss: 2.7382 - 82ms/epoch - 82ms/step
Epoch 103/2000
1/1 - 0s - loss: 2.7234 - val_loss: 2.7299 - 82ms/epoch - 82ms/step
Epoch 104/2000
1/1 - 0s - loss: 2.7143 - val_loss: 2.7253 - 79ms/epoch - 79ms/step
Epoch 105/2000
1/1 - 0s - loss: 2.7090 - val_loss: 2.7208 - 82ms/epoch - 82ms/step
Epoch 106/2000
1/1 - 0s - loss: 2.7046 - val_loss: 2.7129 - 83ms/epoch - 83ms/step
Epoch 107/2000
1/1 - 0s - loss: 2.6950 - val_loss: 2.7055 - 84ms/epoch - 84ms/step
Epoch 108/2000
1/1 - 0s - loss: 2.6871 - val_loss: 2.7003 - 81ms/epoch - 81ms/step
Epoch 109/2000
1/1 - 0s - loss: 2.6823 - val_loss: 2.6959 - 81ms/epoch - 81ms/step
Epoch 110/2000
1/1 - 0s - loss: 2.6757 - val_loss: 2.6860 - 80ms/epoch - 80ms/step
Epoch 111/2000
1/1 - 0s - loss: 2.6670 - val_loss: 2.6787 - 83ms/epoch - 83ms/step
Epoch 112/2000
1/1 - 0s - loss: 2.6590 - val_loss: 2.6742 - 83ms/epoch - 83ms/step
Epoch 113/2000
1/1 

Epoch 200/2000
1/1 - 0s - loss: 2.2850 - val_loss: 2.3026 - 84ms/epoch - 84ms/step
Epoch 201/2000
1/1 - 0s - loss: 2.2825 - val_loss: 2.3002 - 83ms/epoch - 83ms/step
Epoch 202/2000
1/1 - 0s - loss: 2.2802 - val_loss: 2.2985 - 85ms/epoch - 85ms/step
Epoch 203/2000
1/1 - 0s - loss: 2.2782 - val_loss: 2.2961 - 82ms/epoch - 82ms/step
Epoch 204/2000
1/1 - 0s - loss: 2.2762 - val_loss: 2.2951 - 81ms/epoch - 81ms/step
Epoch 205/2000
1/1 - 0s - loss: 2.2745 - val_loss: 2.2935 - 79ms/epoch - 79ms/step
Epoch 206/2000
1/1 - 0s - loss: 2.2735 - val_loss: 2.2928 - 80ms/epoch - 80ms/step
Epoch 207/2000
1/1 - 0s - loss: 2.2720 - val_loss: 2.2902 - 91ms/epoch - 91ms/step
Epoch 208/2000
1/1 - 0s - loss: 2.2700 - val_loss: 2.2861 - 87ms/epoch - 87ms/step
Epoch 209/2000
1/1 - 0s - loss: 2.2653 - val_loss: 2.2819 - 82ms/epoch - 82ms/step
Epoch 210/2000
1/1 - 0s - loss: 2.2614 - val_loss: 2.2804 - 83ms/epoch - 83ms/step
Epoch 211/2000
1/1 - 0s - loss: 2.2598 - val_loss: 2.2800 - 84ms/epoch - 84ms/step
Epoc

Epoch 299/2000
1/1 - 0s - loss: 2.1281 - val_loss: 2.1536 - 82ms/epoch - 82ms/step
Epoch 300/2000
1/1 - 0s - loss: 2.1271 - val_loss: 2.1525 - 91ms/epoch - 91ms/step
Epoch 301/2000
1/1 - 0s - loss: 2.1260 - val_loss: 2.1517 - 80ms/epoch - 80ms/step
Epoch 302/2000
1/1 - 0s - loss: 2.1250 - val_loss: 2.1507 - 81ms/epoch - 81ms/step
Epoch 303/2000
1/1 - 0s - loss: 2.1240 - val_loss: 2.1497 - 83ms/epoch - 83ms/step
Epoch 304/2000
1/1 - 0s - loss: 2.1230 - val_loss: 2.1488 - 80ms/epoch - 80ms/step
Epoch 305/2000
1/1 - 0s - loss: 2.1220 - val_loss: 2.1479 - 82ms/epoch - 82ms/step
Epoch 306/2000
1/1 - 0s - loss: 2.1210 - val_loss: 2.1469 - 82ms/epoch - 82ms/step
Epoch 307/2000
1/1 - 0s - loss: 2.1200 - val_loss: 2.1460 - 79ms/epoch - 79ms/step
Epoch 308/2000
1/1 - 0s - loss: 2.1191 - val_loss: 2.1451 - 81ms/epoch - 81ms/step
Epoch 309/2000
1/1 - 0s - loss: 2.1181 - val_loss: 2.1442 - 81ms/epoch - 81ms/step
Epoch 310/2000
1/1 - 0s - loss: 2.1171 - val_loss: 2.1432 - 81ms/epoch - 81ms/step
Epoc

Epoch 398/2000
1/1 - 0s - loss: 2.0542 - val_loss: 2.0869 - 85ms/epoch - 85ms/step
Epoch 399/2000
1/1 - 0s - loss: 2.0537 - val_loss: 2.0864 - 86ms/epoch - 86ms/step
Epoch 400/2000
1/1 - 0s - loss: 2.0532 - val_loss: 2.0859 - 85ms/epoch - 85ms/step
Epoch 401/2000
1/1 - 0s - loss: 2.0526 - val_loss: 2.0855 - 89ms/epoch - 89ms/step
Epoch 402/2000
1/1 - 0s - loss: 2.0521 - val_loss: 2.0851 - 88ms/epoch - 88ms/step
Epoch 403/2000
1/1 - 0s - loss: 2.0516 - val_loss: 2.0847 - 87ms/epoch - 87ms/step
Epoch 404/2000
1/1 - 0s - loss: 2.0512 - val_loss: 2.0842 - 99ms/epoch - 99ms/step
Epoch 405/2000
1/1 - 0s - loss: 2.0507 - val_loss: 2.0838 - 111ms/epoch - 111ms/step
Epoch 406/2000
1/1 - 0s - loss: 2.0502 - val_loss: 2.0834 - 88ms/epoch - 88ms/step
Epoch 407/2000
1/1 - 0s - loss: 2.0497 - val_loss: 2.0830 - 112ms/epoch - 112ms/step
Epoch 408/2000
1/1 - 0s - loss: 2.0492 - val_loss: 2.0826 - 88ms/epoch - 88ms/step
Epoch 409/2000
1/1 - 0s - loss: 2.0487 - val_loss: 2.0822 - 88ms/epoch - 88ms/step


Epoch 496/2000
1/1 - 0s - loss: 2.0178 - val_loss: 2.0591 - 97ms/epoch - 97ms/step
Epoch 497/2000
1/1 - 0s - loss: 2.0175 - val_loss: 2.0591 - 95ms/epoch - 95ms/step
Epoch 498/2000
1/1 - 0s - loss: 2.0173 - val_loss: 2.0588 - 96ms/epoch - 96ms/step
Epoch 499/2000
1/1 - 0s - loss: 2.0170 - val_loss: 2.0587 - 96ms/epoch - 96ms/step
Epoch 500/2000
1/1 - 0s - loss: 2.0167 - val_loss: 2.0585 - 97ms/epoch - 97ms/step
Epoch 501/2000
1/1 - 0s - loss: 2.0165 - val_loss: 2.0583 - 111ms/epoch - 111ms/step
Epoch 502/2000
1/1 - 0s - loss: 2.0162 - val_loss: 2.0581 - 98ms/epoch - 98ms/step
Epoch 503/2000
1/1 - 0s - loss: 2.0160 - val_loss: 2.0580 - 135ms/epoch - 135ms/step
Epoch 504/2000
1/1 - 0s - loss: 2.0157 - val_loss: 2.0578 - 103ms/epoch - 103ms/step
Epoch 505/2000
1/1 - 0s - loss: 2.0155 - val_loss: 2.0577 - 99ms/epoch - 99ms/step
Epoch 506/2000
1/1 - 0s - loss: 2.0152 - val_loss: 2.0576 - 98ms/epoch - 98ms/step
Epoch 507/2000
1/1 - 0s - loss: 2.0150 - val_loss: 2.0573 - 102ms/epoch - 102ms/s

In [9]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [10]:
(np.mean(acc), np.mean(prob))

(1.0, 0.918796)

In [11]:
embed_dim = 100
num_head = 2

callback = keras.callbacks.EarlyStopping(monitor = 'val_loss', patience = 5, restore_best_weights = True)
inputs = layers.Input((x_masked_train.shape[1],), dtype=tf.int64)
word_embeddings = layers.Embedding(N, embed_dim, name="word_embedding")(inputs)
position_embeddings = PositionEmbedding(sequence_length=len(x_masked_train[0]))(word_embeddings)
encoder_output = word_embeddings + position_embeddings

for i in range(5):
    encoder_output = bert_module(encoder_output, encoder_output, encoder_output, embed_dim, num_head, i)

encoder_output = keras.layers.Lambda(lambda x: x[:,:-1,:], name='slice')(encoder_output)
mlm_output = layers.Dense(N, name="mlm_cls", activation="softmax")(encoder_output)
mlm_model = keras.Model(inputs = inputs, outputs = mlm_output)
adam = Adam()
mlm_model.compile(loss='sparse_categorical_crossentropy', optimizer=adam)
history = mlm_model.fit(x_masked_train, y_masked_labels_train,
                        validation_split = 0.5, callbacks = [callback], 
                        epochs=2000, batch_size=5000, 
                        verbose=2)

Epoch 1/2000
1/1 - 4s - loss: 4.0062 - val_loss: 3.5385 - 4s/epoch - 4s/step
Epoch 2/2000
1/1 - 0s - loss: 3.5311 - val_loss: 3.3683 - 375ms/epoch - 375ms/step
Epoch 3/2000
1/1 - 0s - loss: 3.3743 - val_loss: 3.2384 - 325ms/epoch - 325ms/step
Epoch 4/2000
1/1 - 0s - loss: 3.2523 - val_loss: 3.1908 - 308ms/epoch - 308ms/step
Epoch 5/2000
1/1 - 0s - loss: 3.2024 - val_loss: 3.1443 - 325ms/epoch - 325ms/step
Epoch 6/2000
1/1 - 0s - loss: 3.1461 - val_loss: 3.0952 - 311ms/epoch - 311ms/step
Epoch 7/2000
1/1 - 0s - loss: 3.0865 - val_loss: 3.0677 - 399ms/epoch - 399ms/step
Epoch 8/2000
1/1 - 0s - loss: 3.0521 - val_loss: 3.0483 - 327ms/epoch - 327ms/step
Epoch 9/2000
1/1 - 0s - loss: 3.0308 - val_loss: 3.0379 - 320ms/epoch - 320ms/step
Epoch 10/2000
1/1 - 0s - loss: 3.0223 - val_loss: 3.0383 - 372ms/epoch - 372ms/step
Epoch 11/2000
1/1 - 0s - loss: 3.0258 - val_loss: 3.0377 - 401ms/epoch - 401ms/step
Epoch 12/2000
1/1 - 1s - loss: 3.0276 - val_loss: 3.0303 - 541ms/epoch - 541ms/step
Epoch 1

In [12]:
acc = []
prob = []
x_test_subset = x_masked_test[np.random.choice(x_masked_test.shape[0], size=1000, replace=False)]

for sentence_number in x_test_subset:
    temp = keras.backend.function(inputs = mlm_model.layers[0].input, outputs = mlm_model.layers[-1].output) \
        (np.array(sentence_number).reshape(1,len(sentence_number)))
    temp = temp[:,-1,:]
    acc.append(1 if temp.argmax() == sentence_number[-1] else 0)
    prob.append(temp[0][sentence_number[-1]])

In [13]:
(np.mean(acc), np.mean(prob))

(1.0, 0.985382)